In [1]:
import tensorflow as tf
import pandas as pd
from skimage import io
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
df = pd.read_csv("./sensores.csv")

In [3]:
a = np.random.randn(len(df))

In [4]:
check = (a<0.8)
df_training = df[check] # Me quedo con los registros de mi dataset elegidos como true
df_testing = df[~check] # Me quedo con los elegidos como false

In [5]:
import sklearn

df_training = sklearn.utils.shuffle(df_training)
df_testing = sklearn.utils.shuffle(df_testing)

In [6]:
y_train = df_training.pop("target")
y_test = df_testing.pop("target")

In [7]:
dim_input = 10
n_categories = 5

In [8]:
def getArrayLabels(array):
    y_train_labels = np.array([])
    y_train_labels = np.zeros((len(array), n_categories))
    
    for target in enumerate(array):
        y_train_labels[target[0]][target[1]] = 1
        
    return y_train_labels

In [9]:
y_train_labels = getArrayLabels(y_train)
y_test_labels = getArrayLabels(y_test)

In [10]:
# Creo un arreglo con los datos normalizados
features = ['ax', 'ay', 'az', 'gx', 'gy', 'gz', 'd1', 'd2', 'd3', 'd4']

x_train = df_training.loc[:, features].values
x_train = StandardScaler().fit_transform(x_train)

x_test = df_testing.loc[:, features].values
x_test = StandardScaler().fit_transform(x_test)

In [11]:
x=tf.placeholder(tf.float32,[None,dim_input]) #imagen del numero descompuesta a un vector
P=tf.Variable(tf.zeros([dim_input,n_categories])) #Matriz de pesos, 784 para recibir la imagen, 10 por las posible salidas
b=tf.Variable(tf.zeros([n_categories])) #Vector con bias
y=tf.matmul(x,P)+b #La operacion que se hara en los nodos que reciben entradas
yR=tf.placeholder(tf.float32,[None,n_categories]) # Matriz con las etiquetas REALES del set de datos

In [12]:
softmax=tf.nn.softmax_cross_entropy_with_logits(labels=yR,logits=y)
costo=tf.reduce_mean(softmax)
optimizador=tf.train.GradientDescentOptimizer(0.5).minimize(costo)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [13]:

prediccion = tf.equal(tf.argmax(y, 1), tf.argmax(yR, 1)) #Nos da arreglo de booleanos para decirnos 
                                                         #cuales estan bien y cuales no
accuracy = tf.reduce_mean(tf.cast(prediccion, tf.float32))#Nos da el porcentaje sobre el arreglo de prediccion
Produccion = tf.argmax(y,1)
init=tf.global_variables_initializer()

In [14]:
#Funcion que usaremos para ver que tan bien va a aprendiendo nuestro modelo 
def avance(epoca_i, sess, last_features, last_labels):
    costoActual = sess.run(costo,feed_dict={x: last_features, yR: last_labels})
    Certeza = sess.run(accuracy,feed_dict={x:x_test,yR: y_test_labels})
    print('Epoca: {:<4} - Costo: {:<8.3} Certeza: {:<5.3}'.format(epoca_i,costoActual,Certeza))

In [15]:
n_muestras = 150
n_epocas = 200

with tf.Session() as sess:
    sess.run(init)
    for epoca_i in range(n_epocas):
        offset = (epoca_i * n_muestras) % (y_train_labels.shape[0] - n_muestras)
        batch_x = x_train[offset:(offset + n_muestras), :]
        batch_y = y_train_labels[offset:(offset + n_muestras)]
        sess.run(optimizador, feed_dict={x: batch_x, yR: batch_y})
        if (epoca_i%50==0):
            avance(epoca_i, sess, batch_x, batch_y)
    print('RESULTADO FINAL: ',sess.run(accuracy, feed_dict={x: x_test,yR: y_test_labels}))
    print ('Resultado de una imagen',sess.run(Produccion,feed_dict={x: x_train[1].reshape(1,10)}))

Epoca: 0    - Costo: 1.19     Certeza: 0.806
Epoca: 50   - Costo: 0.178    Certeza: 0.984
Epoca: 100  - Costo: 0.126    Certeza: 0.984
Epoca: 150  - Costo: 0.0835   Certeza: 0.985
Epoca: 200  - Costo: 0.0699   Certeza: 0.985
RESULTADO FINAL:  0.9854881
Resultado de una imagen [0]


In [16]:
x_train[1].reshape(1,10)

array([[ 0.4692018 , -0.62952065, -1.41018834, -0.39631223, -0.0136433 ,
         0.80215451,  1.0652385 ,  0.99643334,  1.35363351,  1.73560247]])

In [17]:
y_train_labels[1]

array([1., 0., 0., 0., 0.])